In [6]:
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor

In [13]:
#data download

In [7]:
train=datasets.FashionMNIST(train=True,download=True,root="data",transform=ToTensor())

100%|██████████| 26421880/26421880 [00:17<00:00, 1544127.10it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 108692.11it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:04<00:00, 905543.51it/s] 


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [8]:
test=datasets.FashionMNIST(train=False,root="data",transform=ToTensor(),download=True)

In [30]:
len(train.classes)

10

In [14]:
#data loader -batching ,parrelell processing.

In [15]:
from torch.utils.data import DataLoader

In [18]:
train_set=DataLoader(train,batch_size=64)
test_set=DataLoader(test,batch_size=64)

In [20]:
for x,y in train_set:
    print(x.shape,y.shape)
    break

torch.Size([64, 1, 28, 28]) torch.Size([64])


In [21]:
for x,y in test_set:
    print(x.shape,y.shape)
    break

torch.Size([64, 1, 28, 28]) torch.Size([64])


In [23]:
torch.cuda.is_available()

False

In [25]:
#Neural network creation

In [26]:
device="cpu"

In [35]:
class neuarlNetwork(nn.Module):
    def __init__(self):
        super(neuarlNetwork,self).__init__()
        self.flatten=nn.Flatten()
        self.linear_stack=nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
        
    
    def forward(self,x):
        x=self.flatten(x)
        logits=self.linear_stack(x)
        return logits
        
model=neuarlNetwork().to(device)
model

neuarlNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [36]:
model.parameters()

<generator object Module.parameters at 0x00000246F1B74660>

In [40]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters())

In [43]:
def trainning_process(model,dataloader,loss_fn,optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch,X,y in enumerate(dataloader):
        X,y=X.to(device),y.to(device)
        pred=model(X)
        loss=loss_fn(pred,y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch%100==0:
            loss=loss.item()
            current=batch*X
            print(f"loss{loss},[{current/size}]")

In [45]:
def test_process(model,dataloader,loss_fn):
    size=len(dataloader.dataset)
    batch=len(dataloader)
    test_loss,accuracy=0,0
    model.eval()
    with torch.no_grad():
        for batch,X,y in enumerate(dataloader):
            X,y=X.to(device),y.to(device)
            pred=model(X)
            test_loss+=loss_fn(pred,y).item()
            correct+=(pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")    

In [48]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_set, model, loss_fn, optimizer)
    test(test_set, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


TypeError: 'FashionMNIST' object is not callable